This script calculates and plots the advection of moisture in moisture space for the DYAMOND models

In [ ]:
from importlib import reload
import numpy as np
import os
import random
import pickle
import functools
import seaborn
import logging
import typhon
import filenames
from pylab import cm
from os.path import join
from netCDF4 import Dataset
from tqdm import tqdm_notebook
from scipy.interpolate import interp1d, interp2d
from scipy.signal import detrend
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import postprocessing_tools as ptools
from moisture_space import utils, plots, moisture_space
mpl_logger = logging.getLogger('matplotlib') 
mpl_logger.setLevel(logging.WARNING)

plt.style.use(typhon.plots.styles.get('typhon'))  

# Config

In [ ]:
models = ['ARPEGE']
runs = {
    'ICON': ['2.5km'],
    'NICAM': ['3.5km'],
    'SAM': ['4.0km'],
    'UM': ['5.0km'],
    'FV3': ['3.25km'],
    'GEOS': ['3.0km'],
    'IFS': ['4.0km'],
    'MPAS': ['3.75km'],
    'ARPEGE': ['2.5km'],
    'ERA5': ['31.0km']
}
exps = []
for m in models:
    for run in runs[m]:
        exps.append(m+'-'+run)
        
time_period = ['2016-08-10', '2016-09-08']
variables_3D = ['TEMP', 'PRES', 'QV', 'RH', 'QC', 'QI', 'U', 'V', 'W', 'A_QV_h',
                'A_QV_v', 'A_RH_h', 'A_RH_v', 'DRH_Dt_h', 'DRH_Dt_v']
variables_2D = []
variables = variables_3D + variables_2D
data_dir = '/mnt/lustre02/work/mh1126/m300773/DYAMOND'
num_samples = 10_000_000
num_percs = 50
perc_values = np.arange(2., 100.5, 2.0)

height = np.arange(100, 17600, 200) # height vector to interpolate all models on

# Read data

In [ ]:
reload(moisture_space)
reload(filenames)
ms = {}
#FIXME
for m in models:#+[ref_model]:
    for run in runs[m]:
        exp = m+'-'+run
        ms[exp] = {}
        for var in variables:
            filename = filenames.averaged_profiles(data_dir, m, run, var, num_samples, num_percs, time_period, '')
            ms[exp][var] = moisture_space.PercMoistureSpace.from_netcdf(filename)
            if var in variables_3D:
                ms[exp][var] = ms[exp][var].interpolate(height, kind='linear', interp_space='linear')
perc_values = ms[exp][var].bins
            

# Calculate additional variables

In [ ]:
reload(moisture_space)
additional_variables = ['QV_sat', 'E_sat', 'RHO', 'VMR', 'dES_dT', 'dT_dp_h']
gamma = 1.4
exp_adiabat = (gamma - 1) / gamma
for exp in exps:
    # saturation specific humidity
    q_sat = utils.rel_hum2spec_hum(
        np.ones((num_percs, len(height))), # RH = 1 everywhere
        ms[exp]['TEMP'].mean,
        ms[exp]['PRES'].mean)
    # saturation water vapour pressure
    e_sat = typhon.physics.e_eq_mixed_mk(
        ms[exp]['TEMP'].mean 
    )
    # density (of moist air)
    density = utils.calc_density_moist_air(
        ms[exp]['PRES'].mean,
        ms[exp]['TEMP'].mean,
        ms[exp]['QV'].mean
    )
    # volume mixing ratio
    vmr = typhon.physics.specific_humidity2vmr(
        ms[exp]['QV'].mean
    )
    # wind speed
    # speed = np.sqrt(ms_perc[exp]['U'].mean ** 2 + ms_perc[exp]['V'].mean ** 2)

    # terms for advection
    cc = e_sat * typhon.constants.heat_of_vaporization / typhon.constants.gas_constant_water_vapor / (ms[exp]['TEMP'].mean ** 2)
    dp_h = np.gradient(ms[exp]['PRES'].mean, bin_size, axis=0)
    dT_dp_h = ms[exp]['TEMP'].mean * (((ms[exp]['PRES'].mean + dp_h) / ms[exp]['PRES'].mean) ** exp_adiabat - 1) / dp_h
    tot_q = ms[exp]['QC'].mean + ms[exp]['QI'].mean
    #grad = np.sqrt(ms_perc[exp]['dRH_dx'].mean ** 2 + ms_perc[exp]['dRH_dy'].mean ** 2)
    
    # save variables as moisture space objects
    ms[exp]['QV_sat'] = moisture_space.PercMoistureSpace.from_mean(q_sat, 'Q_sat', exp, perc_values, height)
    ms[exp]['E_sat'] = moisture_space.PercMoistureSpace.from_mean(e_sat, 'E_sat', exp, perc_values, height)
    ms[exp]['RHO'] = moisture_space.PercMoistureSpace.from_mean(density, 'RHO', exp, perc_values, height)
    ms[exp]['VMR'] = moisture_space.PercMoistureSpace.from_mean(vmr, 'VMR', exp, perc_values, height)
    ms[exp]['dES_dT'] = moisture_space.PercMoistureSpace.from_mean(cc, 'dES_dT', exp, perc_values, height)
    ms[exp]['dT_dp_h'] = moisture_space.PercMoistureSpace.from_mean(dT_dp_h, 'dT_dp_h', exp, perc_values, height)
    
dry_lapse_rate = -typhon.constants.g / typhon.constants.isobaric_mass_heat_capacity                

# Create dictionaries containing fields from all models

In [ ]:
reload(moisture_space)
ms_series = {}
ms_a = {} # dictionary containing model anomalies with respect to multi-model mean
ms_a_series = {} # dictionary containing series of model anomalies
for var in variables_3D+additional_variables:
    perc_space_list = [ms[exp][var] for exp in exps]
    ms_series[var] = moisture_space.MoistureSpaceSeries(perc_space_list)


for var in variables_3D+additional_variables:
    ms_a[var] = {}
    for exp in exps:
        ms_a[var][exp] = moisture_space.PercMoistureSpace.from_mean(
            ms[exp][var].mean - ms_series[var].mean,
            var, exp, perc_values, height)

for var in variables_3D+additional_variables:
    perc_space_list = [ms_a[var][exp] for exp in exps]
    ms_a_series[var] = moisture_space.MoistureSpaceSeries(perc_space_list)
        

# Calculate tendencies of specific and relative humidity in moisture space

Tendency of specific humidity:
\begin{equation*}
\frac{\partial q}{\partial t} = - \vec{v} \nabla{q}
\end{equation*}

Advection of specific humidity can be splitted into (multi-model) mean values and anomalies for each model:
\begin{equation*}
\vec{v} \nabla{q} = \overline{\vec{v}} \nabla{\overline{q}} + \overline{\vec{v}} \nabla{q'} + \vec{v}' \nabla{\overline{q}} + \vec{v}' \nabla{q'}
\end{equation*}

Tendency of relative humidity:
\begin{equation*}
\frac{\partial r}{\partial t} = \frac{dr}{dt} - \vec{v}\nabla{r} + S = (\frac{r}{p} - \frac{L_v r}{R_v T^2} \frac{dT}{dp}) \vec{v} \nabla{p} - \vec{v}\nabla{r} + S =  (-\rho g \frac{r}{p} + \frac{L_v r}{R_v T^2} \Gamma) w - w \frac{\partial r}{\partial z} + (\frac{r}{p} + \frac{L_v r}{R_v \rho c_p T^2}) \vec{v_h} \nabla_h{p} - \vec{v_h} \nabla_h{r} + S
\end{equation*}


\begin{equation*}
(\frac{x}{e_s} \frac{\partial p}{\partial z} - r \Gamma \frac{L_v}{R_v T^2}) w - w \frac{\partial r}{\partial z} + \frac{x}{e_s} \vec{v_h} \nabla_h{p} - r \frac{L_v}{R_v T^2} \frac{dT}{dp} \vec{v_h} \nabla_h{p} - \vec{v_h} \nabla_h{r} + S = 0
\end{equation*}


In [ ]:
reload(moisture_space)
transport_terms_q = [
    'v_grad_q_h_tot',
    'v_grad_q_v', # advection of q with circulation in vertical direction
    'v_grad_q_v_tot',
    'tot', # total transport (vertical, from individual profiles + horizontal)
    'stat' # statistical transport (difference total transport minus v_grad_q_v)
]

transport_terms_r = [
    'v_grad_r_v', # advection of RH with mean circulation in vertical direction
    'v_grad_r_h_tot', # advection of RH with total circulation in horizontal direction
    'v_grad_r_v_tot', # advection of RH with total circulation in vertical direction
    'v_grad_r_tot', # advection of RH with total circulation
    'dr_dt_h_tot', # total change in RH following a parcel in horizontal direction (total velocity)
    'dr_dt_v_tot', # total change in RH following a parcel in vertical direction (total velocity)
    'tot_v', # total tendency of RH - vertical component only
    'tot_h', # total tendency of RH - horizontal component only
    'tot' # total tendency of RH
]
    
t_q = {k: {} for k in transport_terms_q}
t_r = {k: {} for k in transport_terms_r}

for exp in exps:
    # Tendencies in specific humidity

    t_q['v_grad_q_v'][exp] = moisture_space.PercMoistureSpace.from_mean(
        -ms[exp]['W'].mean * np.gradient(ms[exp]['QV'].mean, h, axis=1),
        'v_grad_q_v', exp, perc_values, h
    )

    t_q['v_grad_q_v_tot'][exp] = moisture_space.PercMoistureSpace.from_mean(
        ms[exp]['A_QV_v'].mean,
        'v_grad_q_v_tot', exp, perc_values, h
    )

    if 'A_QV_h' in variables_3D:
        t_q['v_grad_q_h_tot'][exp] = moisture_space.PercMoistureSpace.from_mean(
            ms[exp]['A_QV_h'].mean,
            'v_grad_q_h_tot', exp, perc_values, h
        )

    t_q['tot'][exp] = moisture_space.PercMoistureSpace.from_mean(
        t_q['v_grad_q_v_tot'][exp].mean
        + t_q['v_grad_q_h_tot'][exp].mean,
        'transport_tot', exp, perc_values, h
    )

    t_q['stat'][exp] = moisture_space.PercMoistureSpace.from_mean(
         ms[exp]['A_QV_v'].mean - t_q['v_grad_q_v'][exp].mean,
        'transport_stat', exp, perc_values, h
    )

    # Tendencies in RH

    if 'A_RH_h' in variables_3D:
        t_r['v_grad_r_h_tot'][exp] = moisture_space.PercMoistureSpace.from_mean(
            ms[exp]['A_RH_h'].mean,
            'v_grad_r_h_tot', exp, perc_values, h
        )

    t_r['v_grad_r_v'][exp] = moisture_space.PercMoistureSpace.from_mean(
        -ms[exp]['W'].mean * np.gradient(ms[exp]['RH'].mean, h, axis=1),
        'v_grad_r_v', exp, perc_values, h
    )

    t_r['v_grad_r_v_tot'][exp] = moisture_space.PercMoistureSpace.from_mean(
        ms[exp]['A_RH_v'].mean,
        'v_grad_r_v_tot', exp, perc_values, h
    )

    t_r['v_grad_r_tot'][exp] = moisture_space.PercMoistureSpace.from_mean(
        t_r['v_grad_r_v_tot'][exp].mean
        + t_r['v_grad_r_h_tot'][exp].mean,
        'v_grad_r_tot', exp, perc_values, h
    )

    t_r['dr_dt_v_tot'][exp] = moisture_space.PercMoistureSpace.from_mean(
        ms[exp]['DRH_Dt_v'].mean,
        'dr_dt_v_tot', exp, perc_values, h
    )

    if 'DRH_Dt_h' in variables_3D:
        t_r['dr_dt_h_tot'][exp] = moisture_space.PercMoistureSpace.from_mean(
            ms[exp]['DRH_Dt_h'].mean,
            'dr_dt_h', exp, perc_values, h
        )

    if 'A_RH_h' in variables_3D:
        t_r['tot'][exp] = moisture_space.PercMoistureSpace.from_mean(
            t_r['v_grad_r_h_tot'][exp].mean 
            + t_r['v_grad_r_v_tot'][exp].mean
            + t_r['dr_dt_v_tot'][exp].mean
            + t_r['dr_dt_h_tot'][exp].mean,
            'tot', exp, perc_values, h
        )

    t_r['tot_v'][exp] = moisture_space.PercMoistureSpace.from_mean(
        t_r['v_grad_r_v_tot'][exp].mean
        + t_r['dr_dt_v_tot'][exp].mean,
        'tot_v', exp, perc_values, h
    )

    t_r['tot_h'][exp] = moisture_space.PercMoistureSpace.from_mean(
        t_r['v_grad_r_h_tot'][exp].mean
        + t_r['dr_dt_h_tot'][exp].mean,
        'tot_h', exp, perc_values, h
    )
               
ts_q = {} # dictionary containing series of spec. hum. tendencies of all models
ta_q = {} # dictionary containing anomalies of spec. hum. tendencies for each model
ts_r = {} # dictionary containing series of RH tendencies of all models
ta_r = {} # dictionary containing anomalies of RH tendencies for each model
tas_q = {} # dictionary containing series of spec. hum. anomalies
tas_r = {} # dictionary containing series of RH anomalies
for term in transport_terms_q:
    if term not in ['vm_grad_qm_h', 'vm_grad_qm_v']:
        tas_q[term] = {}
        space_list_q = [t_q[term][exp] for exp in exps]
        ts_q[term] = moisture_space.MoistureSpaceSeries(space_list_q)
        ta_q[term] = {}
        for exp in exps:
            ta_q[term][exp] = moisture_space.PercMoistureSpace.from_mean(
                t_q[term][exp].mean - ts_q[term].mean,
                term, exp, perc_values, h
            )
        space_list_qa = [ta_q[term][exp] for exp in exps]
        tas_q[term] = moisture_space.MoistureSpaceSeries(space_list_qa)
                
for term in transport_terms_r:
    tas_r[term] = {}
    space_list_r = [t_r[term][exp] for exp in exps]
    ts_r[term] = moisture_space.MoistureSpaceSeries(space_list_r)
    ta_r[term] = {}
    for exp in exps:
        ta_r[term][exp] = moisture_space.PercMoistureSpace.from_mean(
                t_r[term][exp].mean - ts_r[term].mean,
                term, exp, perc_values, h
        )
    space_list_ra = [ta_r[term][exp] for exp in exps]
    tas_r[term] = moisture_space.MoistureSpaceSeries(space_list_ra) 


# Tendency of RH and QV - Individual terms

In [ ]:
exp = 'ARPEGE-2.5km'
plt.rcParams.update({'font.size': 17})
num_cols = 3
num_rows = 1
fig, ax = plt.subplots(num_rows, num_cols, figsize=(15, 3. * num_rows))
contours = np.arange(-100., 101., 5.)
x_lims = [0, 100]
y_lims = [0, 17.5]
im = plots.moisture_space_contourf(fig, ax[0], perc_values-0.5, height * 1e-3, (t_r['tot_v'][exp].mean.T) * 1e2 * 24 * 60 * 60, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r', vmin=contours[0], vmax=contours[-1])

im = plots.moisture_space_contourf(fig, ax[1], perc_values-0.5, height * 1e-3, (t_r['tot_h'][exp].mean.T) * 1e2 * 24 * 60 * 60, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r', vmin=contours[0], vmax=contours[-1])

im = plots.moisture_space_contourf(fig, ax[2], perc_values-0.5, height * 1e-3, -t_r['tot'][exp].mean.T * 1e2 * 24 * 60 * 60, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r', vmin=contours[0], vmax=contours[-1])


#im = plots.moisture_space_contourf(fig, ax[3], perc_values-0.5, h * 1e-3, ms_perc[exp][b]['DRH_Dt_c'].mean.T * 1e2 * 24 * 60 * 60, contours,\
#                          x_lims, y_lims, '', '', '', cb=False,
#                          cm_orientation='vertical', cb_extend='both',\
#                          cmap='difference_r', vmin=contours[0], vmax=contours[-1])

ax[0].set_ylabel('Height / km')
for i in range(num_cols):
    ax[i].set_xlabel('IWV percentile')
#ax[0].set_title(r'$\frac{dr}{dt} - w \frac{\partial r}{\partial z}$')
#ax[1].set_title(r'$- u \frac{\partial r}{\partial x}$')
#ax[2].set_title(r'Sum')
cb_ax = fig.add_axes([0.94, 0.12, 0.015, 0.77])
fig.colorbar(im, cax=cb_ax, orientation='vertical', label='RH tendency / (% d$^{-1}$)')
plt.savefig('plots_seminar/RH_tendencies.pdf', bbox_inches='tight', dpi=450)

Multi-model mean

In [ ]:
plt.rcParams.update({'font.size': 17})
num_cols = 3
num_rows = 1
fig, ax = plt.subplots(num_rows, num_cols, figsize=(17, 3.5 * num_rows))
contours = np.arange(-100., 101., 5.)
x_lims = [1, 100]
y_lims = [0, 17.5]
im = plots.moisture_space_contourf(fig, ax[0], perc_values-0.5, h * 1e-3, (ts_r['tot_v'].mean.T) * 1e2 * 24 * 60 * 60, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r', vmin=contours[0], vmax=contours[-1])

im = plots.moisture_space_contourf(fig, ax[1], perc_values-0.5, h * 1e-3, (ts_r['tot_h'].mean.T) * 1e2 * 24 * 60 * 60, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r', vmin=contours[0], vmax=contours[-1])

im = plots.moisture_space_contourf(fig, ax[2], perc_values-0.5, h * 1e-3, (ts_r['tot'].mean.T) * 1e2 * 24 * 60 * 60, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r', vmin=contours[0], vmax=contours[-1])

ax[0].set_ylabel('$z$ / km')
for i in range(num_cols):
    ax[i].set_xlabel('IWV percentile')
ax[0].set_title('Vertical transport')
ax[1].set_title('Horizontal transport')
ax[2].set_title('Total transport')
#ax[1].set_title(r'$- u \frac{\partial r}{\partial x}$')
#ax[2].set_title(r'Sum')
cb_ax = fig.add_axes([0.94, 0.12, 0.015, 0.77])
fig.colorbar(im, cax=cb_ax, orientation='vertical', label='RH tendency / (% d$^{-1}$)')
plt.savefig('plots_seminar/RH_tendencies.pdf', bbox_inches='tight', dpi=450)

In [ ]:
plt.rcParams.update({'font.size': 17})
num_cols = 2
num_rows = 1
fig, ax = plt.subplots(num_rows, num_cols, figsize=(11, 3.5 * num_rows))
contours = np.arange(-100., 101., 5.)
x_lims = [0, 100]
y_lims = [0, 17.5]
im = plots.moisture_space_contourf(fig, ax[0], perc_values-0.5, h * 1e-3, (t_r['tot_v'][exp].mean.T) * 1e2 * 24 * 60 * 60, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r', vmin=contours[0], vmax=contours[-1])

im = plots.moisture_space_contourf(fig, ax[1], perc_values-0.5, h * 1e-3, (t_r['tot_h'][exp].mean.T) * 1e2 * 24 * 60 * 60, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r', vmin=contours[0], vmax=contours[-1])

ax[0].set_ylabel('Height / km')
for i in range(num_cols):
    ax[i].set_xlabel('IWV percentile')
ax[0].set_title('Vertical transport')
ax[1].set_title('Horizontal transport')
#ax[1].set_title(r'$- u \frac{\partial r}{\partial x}$')
#ax[2].set_title(r'Sum')
cb_ax = fig.add_axes([0.94, 0.12, 0.015, 0.77])
fig.colorbar(im, cax=cb_ax, orientation='vertical', label='RH tendency / % d$^{-1}$')
plt.savefig('plots_seminar/RH_tendencies.pdf', bbox_inches='tight', dpi=450)

In [ ]:
exp = 'ARPEGE-2.5km'
plt.rcParams.update({'font.size': 17})
num_cols = 3
num_rows = 1
fig, ax = plt.subplots(num_rows, num_cols, figsize=(15, 3. * num_rows))
contours = np.arange(-100, 101, 10)
x_lims = [0, 100]
y_lims = [0, 17.5]

im = plots.moisture_space_contourf(fig, ax[0], perc_values, h * 1e-3, (t_q['v_grad_q_v_tot'][exp].mean.T) / ms[exp]['QV'].mean.T * 1e2 * 60 * 60 * 24, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r')

im = plots.moisture_space_contourf(fig, ax[1], perc_values, h * 1e-3, t_q['v_grad_q_h_tot'][exp].mean.T / ms[exp]['QV'].mean.T * 1e2 * 60 * 60 * 24, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r')
    
im = plots.moisture_space_contourf(fig, ax[2], perc_values, h * 1e-3, -(t_q['v_grad_q_v_tot'][exp].mean.T + t_q['v_grad_q_h_tot'][exp].mean.T) / ms[exp]['QV'].mean.T * 1e2 * 60 * 60 * 24, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r')
    
ax[0].set_ylabel('Height / km')
for i in range(num_cols):
    ax[i].set_xlabel('IWV percentile')
ax[0].set_title(r'$-w \frac{\partial q}{\partial z} / q$')
ax[1].set_title(r'$-u \frac{\partial q}{\partial x} / q$')
ax[2].set_title('Sum')
cb_ax = fig.add_axes([0.25, -0.2, 0.5, 0.05])
fig.colorbar(im, cax=cb_ax, orientation='horizontal', label='q tendency / q   / % d-1')

In [ ]:
exp = 'ARPEGE-2.5km'
plt.rcParams.update({'font.size': 17})
num_cols = 3
num_rows = 1
fig, ax = plt.subplots(num_rows, num_cols, figsize=(15, 3. * num_rows))
contours = np.arange(-0.01, 0.01, 0.0001)
x_lims = [0, 100]
y_lims = [0, 17.5]

im = plots.moisture_space_contourf(fig, ax[0], perc_values, height * 1e-3, (t_q['v_grad_q_v_tot'][exp].mean.T) * 60 * 60 * 24, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r')

im = plots.moisture_space_contourf(fig, ax[1], perc_values, height * 1e-3, t_q['v_grad_q_h_tot'][exp].mean.T * 60 * 60 * 24, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r')
    
im = plots.moisture_space_contourf(fig, ax[2], perc_values, height* 1e-3, -(t_q['v_grad_q_v_tot'][exp].mean.T + t_q['v_grad_q_h_tot'][exp].mean.T) * 60 * 60 * 24, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r')
    
ax[0].set_ylabel('Height / km')
for i in range(num_cols):
    ax[i].set_xlabel('IWV percentile')
ax[0].set_title(r'$-w \frac{\partial q}{\partial z}$')
ax[1].set_title(r'$-u \frac{\partial q}{\partial x}$')
ax[2].set_title('Sum')
cb_ax = fig.add_axes([0.25, -0.2, 0.5, 0.05])
fig.colorbar(im, cax=cb_ax, orientation='horizontal', label='q tendency / q   / d-1')

All models

In [ ]:
plt.rcParams.update({'font.size': 13})
num_cols = 3
num_rows = max(len(exps), 2)
fig, ax = plt.subplots(num_rows, num_cols, figsize=(14, 3. * num_rows))
contours = np.arange(-100, 101, 5.)
x_lims = [0, 100]
y_lims = [0, 17.5]
for i, exp in enumerate(exps):
    ax[i, 0].annotate(exp, (5, 15))
    im = plots.moisture_space_contourf(fig, ax[i, 0], perc_values, height * 1e-3, t_r['tot_v'][exp].mean.T * 1e2 * 24 * 60 * 60, contours,\
                              x_lims, y_lims, '', '', '', cb=False,
                              cm_orientation='vertical', cb_extend='both',\
                              cmap='difference_r', vmin=contours[0], vmax=contours[-1])
#     im = plots.moisture_space_contourf(fig, ax[i, 1], perc_values, h * 1e-3, t_r['dr_dt_h'][b][exp].mean.T * 24 * 60 * 60, contours,\
#                               x_lims, y_lims, '', '', '', cb=False,
#                               cm_orientation='vertical', cb_extend='both',\
#                               cmap='difference', vmin=contours[0], vmax=contours[-1])
    im = plots.moisture_space_contourf(fig, ax[i, 1], perc_values, height * 1e-3, t_r['tot_h'][exp].mean.T * 1e2 * 24 * 60 * 60, contours,\
                              x_lims, y_lims, '', '', '', cb=False,
                              cm_orientation='vertical', cb_extend='both',\
                              cmap='difference_r', vmin=contours[0], vmax=contours[-1])
    im = plots.moisture_space_contourf(fig, ax[i, 2], perc_values, height * 1e-3, -t_r['tot'][exp].mean.T * 1e2 * 24 * 60 * 60, contours,\
                              x_lims, y_lims, '', '', '', cb=False,
                              cm_orientation='vertical', cb_extend='both',\
                              cmap='difference_r', vmin=contours[0], vmax=contours[-1])
    

    ax[i, 0].set_ylabel('Height / km')
for i in range(num_cols):
    ax[-1, i].set_xlabel('IWV percentile')
ax[0, 0].set_title(r'Vertical transport')
ax[0, 1].set_title(r'Horizontal transport')
ax[0, 2].set_title('Residual')
#ax[0, 1].set_title(r'$- w \frac{\partial r}{\partial z}$')
#ax[0, 3].set_title(r'Sum')
cb_ax = fig.add_axes([0.10, 0.02, 0.35, 0.02])
fig.colorbar(im, cax=cb_ax, orientation='horizontal', label='RH tendency / (% d-1)')

Anomalies in transport 

In [ ]:
plt.rcParams.update({'font.size': 15})
num_cols = 7
num_rows = len(exps)
fig, ax = plt.subplots(num_rows, num_cols, figsize=(20, 3. * num_rows))
contours_rh = np.arange(-15, 15.1, 2.5)
contours = np.arange(-30., 31., 5.)
contours_w = np.arange(-0.005, 0.0051, 0.001)
contours_u = np.arange(-6., 6.01, 1.)
contours_grad = np.arange(-0.00001, 0.00001, 0.000001)
x_lims = [0, 100]
y_lims = [0, 17.5]

for i, exp in enumerate(exps):
    ax[i, 0].annotate(exp, (5, 15))
    im_rh = plots.moisture_space_contourf(fig, ax[i, 0], perc_values-0.5, height * 1e-3, (ms_a['RH'][exp].mean.T) * 1e2, contours_rh,\
                              x_lims, y_lims, '', '', '', cb=False,
                              cm_orientation='vertical', cb_extend='both',\
                              cmap='difference_r', vmin=contours_rh[0], vmax=contours_rh[-1])
    
    im = plots.moisture_space_contourf(fig, ax[i, 1], perc_values-0.5, height * 1e-3, (ta_r['tot'][exp].mean.T) * 1e2 * 24 * 60 * 60, contours,\
                              x_lims, y_lims, '', '', '', cb=False,
                              cm_orientation='vertical', cb_extend='both',\
                              cmap='difference_r', vmin=contours[0], vmax=contours[-1])
    plots.moisture_space_contour(fig, ax[i, 1], perc_values-0.5, h * 1e-3, (ms_a['RH'][exp].mean.T) * 1e2, contours_rh,\
                              x_lims, y_lims, '', '', color='k', lw=0.1)
    
    im = plots.moisture_space_contourf(fig, ax[i, 2], perc_values-0.5, height * 1e-3, (ta_r['tot_v'][exp].mean.T) * 1e2 * 24 * 60 * 60, contours,\
                              x_lims, y_lims, '', '', '', cb=False,
                              cm_orientation='vertical', cb_extend='both',\
                              cmap='difference_r', vmin=contours[0], vmax=contours[-1])
    
    im = plots.moisture_space_contourf(fig, ax[i, 3], perc_values-0.5, height * 1e-3, (ta_r['tot_h'][exp].mean.T) * 1e2 * 24 * 60 * 60, contours,\
                              x_lims, y_lims, '', '', '', cb=False,
                              cm_orientation='vertical', cb_extend='both',\
                              cmap='difference_r', vmin=contours[0], vmax=contours[-1])
    
    im_w = plots.moisture_space_contourf(fig, ax[i, 4], perc_values-0.5, height * 1e-3, (ms_a['W'][exp].mean.T), contours_w,\
                              x_lims, y_lims, '', '', '', cb=False,
                              cm_orientation='vertical', cb_extend='both',\
                              cmap='difference_r', vmin=contours_w[0], vmax=contours_w[-1])
    
    im_u = plots.moisture_space_contourf(fig, ax[i, 5], perc_values-0.5, height * 1e-3, (ms_a['UV'][exp].mean.T), contours_u,\
                              x_lims, y_lims, '', '', '', cb=False,
                              cm_orientation='vertical', cb_extend='both',\
                              cmap='difference_r', vmin=contours_u[0], vmax=contours_u[-1])
    
    im_grad = plots.moisture_space_contourf(fig, ax[i, 6], perc_values-0.5, height * 1e-3, (ms_a['gradRH'][exp].mean.T) * 1e2, contours_grad,\
                              x_lims, y_lims, '', '', '', cb=False,
                              cm_orientation='vertical', cb_extend='both',\
                              cmap='difference_r', vmin=contours_grad[0], vmax=contours_grad[-1])
    
    ax[i, 0].set_ylabel('Height / km')
for i in range(num_cols):
    ax[-1, i].set_xlabel('IWV percentile')

ax[0, 0].set_title('RH anomaly')
ax[0, 1].set_title('Transport\nanomaly')
ax[0, 2].set_title('Vertical transport\nanomaly')
ax[0, 3].set_title('Horizontal transport\nanomaly')
ax[0, 4].set_title('Vertical velocity\nanomaly')
ax[0, 5].set_title('Horizontal velocity\nanomaly')

cb_ax = fig.add_axes([0.15, 0.05, 0.1, 0.01])
fig.colorbar(im_rh, cax=cb_ax, orientation='horizontal', label='RH anomaly / %')

cb_ax = fig.add_axes([0.3, 0.05, 0.3, 0.01])
fig.colorbar(im, cax=cb_ax, orientation='horizontal', label='RH tendency anomaly / (% d-1)')

cb_ax = fig.add_axes([0.67, 0.05, 0.1, 0.01])
fig.colorbar(im_w, cax=cb_ax, orientation='horizontal', label='w anomaly / (m s-1)')

cb_ax = fig.add_axes([0.8, 0.05, 0.1, 0.01])
fig.colorbar(im_u, cax=cb_ax, orientation='horizontal', label='v$_h$ anomaly / (m s-1)')
plt.savefig('plots_seminar/transport_anomalies.pdf', dpi=450)


# Anomalies in relative humidity and vertical velocity

In [ ]:
num_cols = 2
num_rows = int(np.ceil(len(exps) / num_cols))
fig, ax = plt.subplots(num_rows, num_cols, figsize=(12, 18))
ax = ax.ravel()
contours_rh = np.arange(-15., 15., 1.)
contours_w = [-0.0008, -0.0005, -0.00025, 0.00025, 0.0005, 0.0008]
x_lims = [0, 100]
y_lims = [0, 17.5]
x_label = 'Percentile of IWV'
y_label = 'Height / km'
perc_values_reduced = (perc_values[:-1] + 0.5 * np.diff(perc_values))[::2]
for i, exp in enumerate(exps):
    ax[i].set_title(exp)
    im = plots.moisture_space_contourf(fig, ax[i], perc_values, height * 1e-3, ms_a['RH'][exp].mean.T * 1e2, contours_rh,\
                              x_lims, y_lims, '', '', '', cb=False,
                              cm_orientation='vertical', cb_extend='both',\
                              cmap='BrBG', vmin=contours_rh[0], vmax=contours_rh[-1])
    plots.moisture_space_contour(fig, ax[i], perc_values_reduced, height * 1e-3, ms_a['W'][exp].reduced_bins(25).T, contours_w,\
                              x_lims, y_lims, '', '', color='k')
    #ax[i].contour(perc_values_reduced, h * 1e-3, ms_a['W'][b][exp].reduced_bins(25).T, [0.], color='k', linewidths=3.)
    if i%2 == 0:
        ax[i].set_ylabel(y_label)

# Difference between averaged vertical transport of q and transport with averaged vertical velocity

In [ ]:
plt.rcParams.update({'font.size': 13})
fig, ax = plt.subplots(max(len(exps), 2), 4, figsize=(12, 15))
contours = np.arange(-1, 1.1, 0.1)
x_lims = [0, 100]
y_lims = [0, 17.5]
for i, exp in enumerate(exps):
    ax[i, 0].annotate(exp, (5, 15))
    
    im = plots.moisture_space_contourf(fig, ax[i, 0], perc_values, height * 1e-3, (t_q['v_grad_q_v_tot'][exp].mean.T) / ms[exp]['QV'].mean.T * 60 * 60 * 24, contours,\
                              x_lims, y_lims, '', '', '', cb=False,
                              cm_orientation='vertical', cb_extend='both',\
                              cmap='difference')
    
    im = plots.moisture_space_contourf(fig, ax[i, 1], perc_values, height * 1e-3, (t_q['v_grad_q_v'][exp].mean.T) / ms[exp]['QV'].mean.T * 60 * 60 * 24, contours,\
                              x_lims, y_lims, '', '', '', cb=False,
                              cm_orientation='vertical', cb_extend='both',\
                              cmap='difference')
    
    im = plots.moisture_space_contourf(fig, ax[i, 2], perc_values, height * 1e-3, ts_q['stat'].mean.T / ms[exp]['QV'].mean.T * 60 * 60 * 24, contours,\
                              x_lims, y_lims, '', '', '', cb=False,
                              cm_orientation='vertical', cb_extend='both',\
                              cmap='difference')
    im = plots.moisture_space_contourf(fig, ax[i, 3], perc_values, height * 1e-3, t_q['v_grad_q_h_tot'][exp].mean.T / ms[exp]['QV'].mean.T * 60 * 60 * 24, contours,\
                              x_lims, y_lims, '', '', '', cb=False,
                              cm_orientation='vertical', cb_extend='both',\
                              cmap='difference')
    
    
    
ax[0, 0].set_title('Mean of transport')
ax[0, 1].set_title('Transport by mean circulation')
ax[0, 2].set_title('Difference')
cb_ax = fig.add_axes([0.12, 0.08, 0.35, 0.008])
fig.colorbar(im, cax=cb_ax, orientation='horizontal', label='T/q / d-1')

# Correlation between moisture anomalies and transport anomalies

In [ ]:
reload(moisture_space)

corr = {}
p_val = {}

corr_pairs = [
    (tas_q['v_grad_q_v_tot'], ms_a_series['QV']),
    (tas_q['v_grad_q_h_tot'], ms_a_series['QV']),
    (tas_q['tot'], ms_a_series['QV']),
    (tas_r['tot'], ms_a_series['RH']),
    (tas_r['tot_v'], ms_a_series['RH']),
    (tas_r['tot_h'], ms_a_series['RH']),
    (tas_r['v_grad_r_h_tot'], ms_a_series['RH']),
    (tas_r['v_grad_r_v_tot'], ms_a_series['RH']),
    (tas_r['v_grad_r_tot'], ms_a_series['RH']),
    (tas_r['dr_dt_v_tot'], ms_a_series['RH']),
    (ms_a_series['W'], ms_a_series['RH']),
    (tas_r['dr_dt_v_tot'], ms_a_series['W']),
    (tas_r['v_grad_r_v_tot'], ms_a_series['W']),
    (tas_r['tot_v'], ms_a_series['W']),
    #(tas_r['tot_h'], ms_a_series['UV']),
    #(tas_r['tot_h'], ms_a_series['gradRH']),
    #(ms_a_series['RH'], ms_a_series['gradRH']),
    #(ms_a_series['Qtot'], ms_a_series['QV']),
    #(ms_a_series['UV'], ms_a_series['RH']),
    #(ms_a_series['UV'], ms_a_series['QV']),
    
]

for c in corr_pairs:
    pair = moisture_space.MoistureSpaceSeriesPair(c[0], c[1])
    key = (c[0].variable, c[1].variable)
    corr[key], p_val[key] = pair.correlation(corrtype='pearson')

In [ ]:
#colorbar:

col = typhon.plots.cmap2rgba('difference', 8)
col[3:5,0:3] = [0.9, 0.9, 0.9]#ml_colors.to_rgb('lightgrey')
cmap = ListedColormap(col)

# Total transport
fig, ax = plt.subplots(figsize=(10, 5))
im = ax.contourf(perc_values, h*1e-3, corr[('tot', 'RH')].T, 7, vmin=-1, vmax=1, cmap=cmap)
#ax.contourf(perc_values, h*1e-3, p_val[('tot', 'RH')].T, [0., 0.05], vmin=0, vmax=1, colors='none', hatches='.')
fig.colorbar(im, orientation='vertical', label=r"C(RH', $(\vec{v} \nabla$RH)')", ticks=np.arange(-1, 1.1, 0.25))
ax.set_yticks(np.arange(0, 18, 2.5))
ax.set_xlabel('Percentile of IWV')
ax.set_ylabel('$z$ / km')
plt.savefig('plots_seminar/corr_RHtransport_RH.pdf', bbox_inches='tight', dpi=400)

# Vertical transport
fig, ax = plt.subplots(figsize=(10, 5))
im = ax.contourf(perc_values, h*1e-3, corr[('tot_v', 'RH')].T, 8, vmin=-1, vmax=1, cmap=cmap)
fig.colorbar(im, orientation='vertical', label=r"C(RH', $(w \frac{d \mathrm{RH}}{dz})$')", ticks=np.arange(-1, 1.1, 0.25))
ax.set_yticks(np.arange(0, 18, 2.5))
ax.set_xlabel('Percentile of IWV')
ax.set_ylabel('$z$ / km')
plt.savefig('plots_seminar/corr_verticalRHtransport_RH.pdf', bbox_inches='tight', dpi=400)

# Horizontal transport
fig, ax = plt.subplots(figsize=(10, 5))
im = ax.contourf(perc_values, h*1e-3, corr[('tot_h', 'RH')].T, 7, vmin=-1, vmax=1, cmap=cmap)
fig.colorbar(im, orientation='vertical', label=r"C(RH', $(\vec{v_h} \nabla_h$RH)')", ticks=np.arange(-1, 1.1, 0.25))
ax.set_yticks(np.arange(0, 18, 2.5))
ax.set_xlabel('Percentile of IWV')
ax.set_ylabel('$z$ / km')
plt.savefig('plots_seminar/corr_horizontalRHtransport_RH.pdf', bbox_inches='tight', dpi=400)

In [ ]:
plt.rcParams.update({'font.size': 20})
black_contours = [-0.8, 0.8]
fig, ax = plt.subplots(figsize=(12, 5))
ax.set_title(r'Corr($(\frac{dRH}{dt} - w \frac{\partial RH}{\partial z})\prime, RH \prime$)')
im = ax.contourf(perc_values, h*1e-3, corr[('tot_v', 'RH')].T, 10, vmin=-1, vmax=1, cmap='difference')
#ax.contour(perc_values, h*1e-3, corr[('tot_v', 'RH')].T, black_contours, colors='k')
fig.colorbar(im, orientation='vertical', label='Corr. coefficient', ticks=np.arange(-1, 1.1, 0.2))
ax.set_xlabel('Percentile of IWV')
ax.set_ylabel('Height / km')

fig, ax = plt.subplots(figsize=(12, 5))
ax.set_title(r'Corr($dr/dt \prime, RH \prime$)')
im = ax.contourf(perc_values, h*1e-3, corr[('dr_dt_v_tot', 'RH')].T, 10, vmin=-1, vmax=1, cmap='difference')
#ax.contour(perc_values, h*1e-3, corr[('W', 'RH')].T, black_contours, colors='k')
fig.colorbar(im, orientation='vertical', label='Corr. coefficient', ticks=np.arange(-1, 1.1, 0.2))
ax.set_xlabel('Percentile of IWV')
ax.set_ylabel('Height / km')

fig, ax = plt.subplots(figsize=(12, 5))
ax.set_title(r'Corr($W \prime, RH \prime$)')
im = ax.contourf(perc_values, h*1e-3, corr[('W', 'RH')].T, 10, vmin=-1, vmax=1, cmap='difference')
#ax.contour(perc_values, h*1e-3, corr[('W', 'RH')].T, black_contours, colors='k')
fig.colorbar(im, orientation='vertical', label='Corr. coefficient', ticks=np.arange(-1, 1.1, 0.2))
ax.set_xlabel('Percentile of IWV')
ax.set_ylabel('Height / km')
plt.savefig('plots_seminar/corr_W_RH.pdf', dpi=400)

fig, ax = plt.subplots(figsize=(12, 5))
ax.set_title(r'Corr($(u \frac{\partial RH}{\partial x}) \prime, RH \prime$)')
im = ax.contourf(perc_values, h*1e-3, corr[('v_grad_r_h_tot', 'RH')].T, 10, vmin=-1, vmax=1, cmap='difference')
#ax.contour(perc_values, h*1e-3, corr[('W', 'RH')].T, black_contours, colors='k')
fig.colorbar(im, orientation='vertical', label='Corr. coefficient', ticks=np.arange(-1, 1.1, 0.2))
ax.set_xlabel('Percentile of IWV')
ax.set_ylabel('Height / km')

fig, ax = plt.subplots(figsize=(12, 5))
ax.set_title(r'Corr($(\frac{dRH}{dt} - u \frac{\partial RH}{\partial x}) \prime, RH \prime$)')
im = ax.contourf(perc_values, h*1e-3, corr[('tot_h', 'RH')].T, 10, vmin=-1, vmax=1, cmap='difference')
#ax.contour(perc_values, h*1e-3, corr[('W', 'RH')].T, black_contours, colors='k')
fig.colorbar(im, orientation='vertical', label='Corr. coefficient', ticks=np.arange(-1, 1.1, 0.2))
ax.set_xlabel('Percentile of IWV')
ax.set_ylabel('Height / km')

fig, ax = plt.subplots(figsize=(12, 5))
ax.set_title(r'Corr($(w \frac{\partial q}{\partial z}) \prime, q \prime$)')
im = ax.contourf(perc_values, h*1e-3, corr[('v_grad_q_v_tot', 'QV')].T, 10, vmin=-1, vmax=1, cmap='difference')
#ax.contour(perc_values, h*1e-3, corr[('ta_tot', 'qa')].T, black_contours, colors='k')
fig.colorbar(im, orientation='vertical', label='Corr. coefficient', ticks=np.arange(-1, 1.1, 0.2))
ax.set_xlabel('Percentile of IWV')
ax.set_ylabel('Height / km')

fig, ax = plt.subplots(figsize=(12, 5))
ax.set_title(r'Corr($(u \frac{\partial q}{\partial x}) \prime, q \prime$)')
im = ax.contourf(perc_values, h*1e-3, corr[('v_grad_q_h_tot', 'QV')].T, 10, vmin=-1, vmax=1, cmap='difference')
#ax.contour(perc_values, h*1e-3, corr[('ta_tot', 'qa')].T, black_contours, colors='k')
fig.colorbar(im, orientation='vertical', label='Corr. coefficient', ticks=np.arange(-1, 1.1, 0.2))
ax.set_xlabel('Percentile of IWV')
ax.set_ylabel('Height / km')

fig, ax = plt.subplots(figsize=(12, 5))
ax.set_title(r'Corr($(Transport) \prime, q \prime$)')
im = ax.contourf(perc_values, h*1e-3, corr[('tot', 'QV')].T, 10, vmin=-1, vmax=1, cmap='difference')
#ax.contour(perc_values, h*1e-3, corr[('ta_tot', 'qa')].T, black_contours, colors='k')
fig.colorbar(im, orientation='vertical', label='Corr. coefficient', ticks=np.arange(-1, 1.1, 0.2))
ax.set_xlabel('Percentile of IWV')
ax.set_ylabel('Height / km')

fig, ax = plt.subplots(figsize=(12, 5))
ax.set_title(r'Corr($Vertical transport \prime, W \prime$)')
im = ax.contourf(perc_values, h*1e-3, corr[('tot_v', 'W')].T, 10, vmin=-1, vmax=1, cmap='difference')
#ax.contour(perc_values, h*1e-3, corr[('ta_tot', 'qa')].T, black_contours, colors='k')
fig.colorbar(im, orientation='vertical', label='Corr. coefficient', ticks=np.arange(-1, 1.1, 0.2))
ax.set_xlabel('Percentile of IWV')
ax.set_ylabel('Height / km')

# fig, ax = plt.subplots(figsize=(12, 5))
# ax.set_title(r'Corr($speed \prime, q \prime$)')
# im = ax.contourf(perc_values, h*1e-3, corr[('UV', 'QV')].T, 10, vmin=-1, vmax=1, cmap='difference')
# #ax.contour(perc_values, h*1e-3, corr[('ta_tot', 'qa')].T, black_contours, colors='k')
# fig.colorbar(im, orientation='vertical', label='Corr. coefficient', ticks=np.arange(-1, 1.1, 0.2))
# ax.set_xlabel('Percentile of IWV')
# ax.set_ylabel('Height / km')

# fig, ax = plt.subplots(figsize=(12, 5))
# ax.set_title(r'Corr($Horizontal transport \prime, Horizontal windspeed \prime$)')
# im = ax.contourf(perc_values, h*1e-3, corr[('tot_h', 'UV')].T, np.arange(-1, 1.1, 0.25), vmin=-1, vmax=1, cmap=cmap)
# #ax.contour(perc_values, h*1e-3, corr[('ta_tot', 'qa')].T, black_contours, colors='k')
# fig.colorbar(im, orientation='vertical', label='Corr. coefficient', ticks=np.arange(-1, 1.1, 0.2))
# ax.set_xlabel('Percentile of IWV')
# ax.set_ylabel('Height / km')

# fig, ax = plt.subplots(figsize=(12, 5))
# ax.set_title(r'Corr($Horizontal transport \prime, Horizontal RH gradient \prime$)')
# im = ax.contourf(perc_values, h*1e-3, corr[('tot_h', 'gradRH')].T, 7, vmin=-1, vmax=1, cmap=cmap)
# #ax.contour(perc_values, h*1e-3, corr[('ta_tot', 'qa')].T, black_contours, colors='k')
# fig.colorbar(im, orientation='vertical', label='Corr. coefficient', ticks=np.arange(-1, 1.1, 0.25))
# ax.set_xlabel('Percentile of IWV')
# ax.set_ylabel('Height / km')

# fig, ax = plt.subplots(figsize=(12, 5))
# ax.set_title(r'Corr($RH \prime, Horizontal RH gradient \prime$)')
# im = ax.contourf(perc_values, h*1e-3, corr[('RH', 'gradRH')].T, np.arange(-1, 1.1, 0.25), vmin=-1, vmax=1, cmap=cmap)
# #ax.contour(perc_values, h*1e-3, corr[('ta_tot', 'qa')].T, black_contours, colors='k')
# fig.colorbar(im, orientation='vertical', label='Corr. coefficient', ticks=np.arange(-1, 1.1, 0.25))
# ax.set_xlabel('Percentile of IWV')
# ax.set_ylabel('Height / km')


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
ax[0].set_title(r'Corr($\overline{w} \frac{d q \prime}{dz} + w \prime \frac{d \overline{q}}{dz} + w \prime \frac{d q \prime}{dz}, q \prime$)')
ax[0].contourf(perc_values, h*1e-3, corr[('ta_v', 'qa')].T, 10, vmin=-1, vmax=1, cmap='difference')
ax[1].set_title(r'Corr($\overline{u} \frac{d q \prime}{dx} + u \prime \frac{d \overline{q}}{dx} + u \prime \frac{d q \prime}{dx}, q \prime$)')
ax[1].contourf(perc_values, h*1e-3, corr[('ta_h', 'qa')].T, 10, vmin=-1, vmax=1, cmap='difference')


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
p = [5, 7500]
ind_h = np.argmin(np.abs(perc_values - p[0]))
ind_v = np.argmin(np.abs(h - p[1]))

for i, exp in enumerate(exps):
    ax[0].scatter(ta_r['tot'][exp].mean[ind_h, ind_v], ms_a['RH'][exp].mean[ind_h, ind_v] * 1e2, s=40, color=plots.DYAMOND_colors(exp))
    ax[1].scatter(ms_a['W'][exp].mean[ind_h, ind_v], ms_a['RH'][exp].mean[ind_h, ind_v] * 1e2, s=40, color=plots.DYAMOND_colors(exp))
ax[0].set_xlabel('$TRH$ anomaly / s-1')
ax[0].set_ylabel('RH anomaly / %')
ax[1].set_xlabel('$W$ anomaly / s-1')
ax[1].set_ylabel('RH anomaly / %')
ax[0].set_xlim(-0.0000005, 0.0000005)
ax[1].set_xlim(-0.003, 0.003)

In [ ]:
exp = 'ICON-2.5km'
model = 'ICON'
plt.rcParams.update({'font.size': 17})
num_cols = 5
num_rows = 1
fig, ax = plt.subplots(num_rows, num_cols, figsize=(22, 3. * num_rows))
contours = np.arange(-100., 101., 5.)
x_lims = [0, 100]
y_lims = [0, 17.5]

if model == 'NICAM':
    he = height[exp][3:]
else: 
    he = height[exp][20:]

cond_file = f'/mnt/lustre02/work/mh1126/m300773/DYAMOND/ICON/random_samples/{model}_drdt_cond.pkl'
with open(cond_file, 'rb') as infile:
    icon_cond = pickle.load(infile)

ic = moisture_space.PercMoistureSpace.from_mean(icon_cond, 'cond', 'ICON', perc_values, he).interpolate(h)


im = plots.moisture_space_contourf(fig, ax[0], perc_values-0.5, h * 1e-3, (t_r['tot_v'][exp].mean.T) * 1e2 * 24 * 60 * 60, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r', vmin=contours[0], vmax=contours[-1])

im = plots.moisture_space_contourf(fig, ax[1], perc_values-0.5, h * 1e-3, (t_r['tot_h'][exp].mean.T) * 1e2 * 24 * 60 * 60, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r', vmin=contours[0], vmax=contours[-1])

im = plots.moisture_space_contourf(fig, ax[2], perc_values-0.5, h * 1e-3, (t_r['tot'][exp].mean.T) * 1e2 * 24 * 60 * 60, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r', vmin=contours[0], vmax=contours[-1])

im = plots.moisture_space_contourf(fig, ax[3], perc_values-0.5, h * 1e-3, ic.mean.T * 1e2 * 24 * 60 * 60, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r', vmin=contours[0], vmax=contours[-1])


im = plots.moisture_space_contourf(fig, ax[4], perc_values-0.5, h * 1e-3, -(t_r['tot_v'][exp].mean.T + t_r['tot_h'][exp].mean.T + ic.mean.T) * 1e2 * 24 * 60 * 60, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r', vmin=contours[0], vmax=contours[-1])

ax[0].set_ylabel('Height / km')
for i in range(num_cols):
    ax[i].set_xlabel('IWV percentile')
ax[0].set_title('Vertical transport')
ax[1].set_title('Horizontal transport')
ax[2].set_title('Total transport')
ax[3].set_title('Condensation/Evaporation')
ax[4].set_title('Residual')
#ax[0].set_title(r'$\frac{dr}{dt} - w \frac{\partial r}{\partial z}$')
#ax[1].set_title(r'$- u \frac{\partial r}{\partial x}$')
#ax[2].set_title(r'Sum')
cb_ax = fig.add_axes([0.25, -0.2, 0.5, 0.05])
fig.colorbar(im, cax=cb_ax, orientation='horizontal', label='RH tendency / % d$^{-1}$')
plt.savefig('plots_seminar/RH_tendencies.pdf', dpi=450, bbox_inches='tight')

Importance of condensation term

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
contours = np.arange(-100., 101., 5.)
x_lims = [0, 100]
y_lims = [0, 17.5]
im = ax.pcolor(perc_values, h * 1e-3, ((np.abs(ic.mean.T)) / np.abs(t_r['tot'][exp].mean.T)), cmap=plt.get_cmap('density', 10), vmin=0, vmax=1)

ax.set_yticks(np.arange(0, 18, 2.5))
fig.colorbar(im, label=r'$| (\frac{\partial RH}{\partial t})_{microph.} / (\frac{\partial RH}{\partial t})_{transp.} |$', extend='max')
ax.set_xlabel('Percentile of IWV')
ax.set_ylabel('$z$ / km')
plt.savefig('plots_seminar/fraction_microphysical_tendencies.pdf', dpi=450)

In [ ]:
exp = 'ICON-2.5km'
model = 'ICON'
plt.rcParams.update({'font.size': 17})
num_cols = 5
num_rows = 1
fig, ax = plt.subplots(num_rows, num_cols, figsize=(22, 3. * num_rows))
contours = np.arange(-100, 101, 10)
x_lims = [0, 100]
y_lims = [0, 17.5]

if model == 'NICAM':
    he = height[exp][3:]
else: 
    he = height[exp][20:]

cond_file = f'/mnt/lustre02/work/mh1126/m300773/DYAMOND/ICON/random_samples/{model}_dqdt_cond.pkl'
with open(cond_file, 'rb') as infile:
    icon_cond = pickle.load(infile)
ic = moisture_space.PercMoistureSpace.from_mean(icon_cond, 'cond', 'ICON', perc_values, he).interpolate(h)

im = plots.moisture_space_contourf(fig, ax[0], perc_values, h * 1e-3, (t_q['v_grad_q_v_tot'][exp].mean.T) / ms_perc[exp]['QV'].mean.T * 1e2 * 60 * 60 * 24, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r')

im = plots.moisture_space_contourf(fig, ax[1], perc_values, h * 1e-3, t_q['v_grad_q_h_tot'][exp].mean.T / ms_perc[exp]['QV'].mean.T * 1e2 * 60 * 60 * 24, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r')

im = plots.moisture_space_contourf(fig, ax[2], perc_values, h * 1e-3, t_q['tot'][exp].mean.T / ms_perc[exp]['QV'].mean.T * 1e2 * 60 * 60 * 24, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r')

im = plots.moisture_space_contourf(fig, ax[3], perc_values, h * 1e-3, ic.mean.T / ms_perc[exp]['QV'].mean.T * 1e2 * 60 * 60 * 24, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r')
    
im = plots.moisture_space_contourf(fig, ax[4], perc_values, h * 1e-3, -(t_q['v_grad_q_v_tot'][exp].mean.T + t_q['v_grad_q_h_tot'][exp].mean.T + ic.mean.T) / ms_perc[exp]['QV'].mean.T * 1e2 * 60 * 60 * 24, contours,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='difference_r')
    
ax[0].set_ylabel('Height / km')
for i in range(num_cols):
    ax[i].set_xlabel('IWV percentile')
ax[0].set_title('Vertical transport')
ax[1].set_title('Horizontal transport')
ax[2].set_title('Total transport')
ax[3].set_title('Condensation/Evaporation')
ax[4].set_title('Residual')
cb_ax = fig.add_axes([0.25, -0.2, 0.5, 0.05])
fig.colorbar(im, cax=cb_ax, orientation='horizontal', label='q tendency / q   / % d-1')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
contours = np.arange(-100., 101., 5.)
x_lims = [0, 100]
y_lims = [0, 17.5]
im = ax.pcolor(perc_values, h * 1e-3, ((np.abs(ic.mean.T)) / np.abs(t_q['tot'][exp].mean.T)), cmap=plt.get_cmap('density', 10), vmin=0, vmax=1)

ax.set_yticks(np.arange(0, 18, 2.5))
fig.colorbar(im, label=r'$| (\frac{\partial q}{\partial t})_{microph.} / (\frac{\partial q}{\partial t})_{transp.} |$', extend='max')
ax.set_xlabel('Percentile of IWV')
ax.set_ylabel('$z$ / km')
plt.savefig('plots_seminar/fraction_microphysical_tendencies_q.pdf', dpi=450)

In [ ]:
exp = 'NICAM-3.5km'
plt.rcParams.update({'font.size': 17})
num_cols = 1
num_rows = 1
fig, ax = plt.subplots(num_rows, num_cols, figsize=(11, 3.5 * num_rows))
contours = np.arange(-100., 101., 5.)
x_lims = [0, 100]
y_lims = [0, 17.5]
im = plots.moisture_space_contourf(fig, ax, perc_values-0.5, h * 1e-3, (ms_perc[exp]['QR'].mean.T / ms_perc[exp]['QV'].mean.T) * 100, 100,\
                          x_lims, y_lims, '', '', '', cb=False,
                          cm_orientation='vertical', cb_extend='both',\
                          cmap='density')

ax.set_ylabel('$z$ / km')
ax.set_xlabel('IWV percentile')

fig.colorbar(im, orientation='vertical', label='QR')
#plt.savefig('plots_seminar/RH_tendencies.pdf', bbox_inches='tight', dpi=450)